# Image Inpainting with Diffusion on Simple ShapesThis notebook builds a diffusion-based image inpainting model for simple grayscale shapes. Image inpainting fills in missing regions of an image; here we use a denoising diffusion process together with a small U-Net to reconstruct masked portions. The dataset contains 32×32 single-channel images of circles, squares, and triangles stored as NumPy arrays.

## Table of Contents- [Dataset and preprocessing](#Dataset-and-preprocessing)- [Inpainting mask](#Inpainting-mask)- [Diffusion model and U-Net](#Diffusion-model-and-U-Net)- [Training loop](#Training-loop)- [Inpainting and evaluation](#Inpainting-and-evaluation)

## Dataset and preprocessingImages are loaded from `.npy` files (`circle.npy`, `square.npy`, `triangle.npy`) located in the `data` folder. Each image is grayscale and reshaped to a tensor of shape `(1, 32, 32)`, converted to `float32`, and normalized to the range `[−1, 1]`. An optional limit `max_items_per_class` allows sub-sampling each shape class. A `DataLoader` is created with the configured batch size.

In [ ]:
import osimport mathimport numpy as npimport torchfrom torch import nnfrom torch.utils.data import Dataset, DataLoaderimport torch.nn.functional as Ffrom torchvision.utils import make_gridimport matplotlib.pyplot as pltfrom tqdm.auto import tqdm# ReproducibilitySEED = 42torch.manual_seed(SEED)np.random.seed(SEED)# Paths and hyperparametersDATASET_PATH = "data"IMAGE_SIZE = 32CHANNELS = 1BATCH_SIZE = 64LEARNING_RATE = 1e-3EPOCHS = 30TIMESTEPS = 300MASK_RATIO = 0.5max_items_per_class = None  # set an integer to limit samples per shapeDEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")def to_tensor(img: np.ndarray) -> torch.Tensor:    """Convert a numpy image to a normalized torch tensor in [-1, 1]."""    img = img.astype(np.float32)    # Ensure correct shape    img = img.reshape(1, IMAGE_SIZE, IMAGE_SIZE)    # Normalize to [-1, 1]    img = (img / 127.5) - 1.0    return torch.from_numpy(img)class SimpleShapesDataset(Dataset):    """Dataset loading simple shape numpy arrays and returning tensors."""    def __init__(self, root: str, max_items_per_class=None):        super().__init__()        self.samples = []        for name in ["circle", "square", "triangle"]:            path = os.path.join(root, f"{name}.npy")            data = np.load(path)            if max_items_per_class is not None:                data = data[:max_items_per_class]            self.samples.extend([to_tensor(x) for x in data])        self.samples = torch.stack(self.samples)    def __len__(self):        return len(self.samples)    def __getitem__(self, idx):        return self.samples[idx]# Load dataset and create dataloaderdataset = SimpleShapesDataset(DATASET_PATH, max_items_per_class=max_items_per_class)loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)print(f"Loaded {len(dataset)} samples on {DEVICE}.")

## Inpainting maskWe remove a central square region from each image using a binary mask with value `1` inside the missing area and `0` elsewhere. The `MASK_RATIO` controls the fraction of the image side covered by the square. The same mask is used during training and inference to hide the target region and to guide reconstruction.

In [ ]:
def central_square_mask(image_shape, ratio: float):    """Create a central square mask with ones in the missing region."""    _, h, w = image_shape    mask = torch.zeros(image_shape, dtype=torch.float32)    size_h, size_w = int(h * ratio), int(w * ratio)    start_h = (h - size_h) // 2    start_w = (w - size_w) // 2    mask[:, start_h : start_h + size_h, start_w : start_w + size_w] = 1.0    return maskdef apply_inpainting_mask(imgs: torch.Tensor, ratio: float = MASK_RATIO):    """Apply a central mask to a batch of images and return masked images and mask."""    b, c, h, w = imgs.shape    mask = central_square_mask((c, h, w), ratio).to(imgs.device)    mask = mask.unsqueeze(0).expand(b, -1, -1, -1)    # Zero out the masked region for visualization    masked_imgs = imgs * (1 - mask)    return masked_imgs, mask# Visualize a few masked samplesbatch = next(iter(loader))masked_batch, vis_mask = apply_inpainting_mask(batch)fig, axes = plt.subplots(2, 6, figsize=(12, 4))for i in range(6):    axes[0, i].imshow(batch[i].squeeze(), cmap="gray", vmin=-1, vmax=1)    axes[0, i].axis("off")    axes[0, i].set_title("Original")    axes[1, i].imshow(masked_batch[i].squeeze(), cmap="gray", vmin=-1, vmax=1)    axes[1, i].axis("off")    axes[1, i].set_title("Masked")plt.tight_layout()plt.show()

## Diffusion model and U-NetA Denoising Diffusion Probabilistic Model (DDPM) gradually adds Gaussian noise to an image through a forward process \(q(x_t \mid x_0)\) controlled by a noise schedule. With pre-computed betas \(eta_t\), we derive \(lpha_t = 1 - eta_t\) and the cumulative product \(ar{lpha}_t\). The forward sampling is\[ x_t = \sqrt{ar{lpha}_t} x_0 + \sqrt{1-ar{lpha}_t}\, \epsilon. \]The model learns to predict the noise \(\epsilon\) given a noisy input. We concatenate the masked noisy image with the binary mask and provide a sinusoidal time embedding to a compact U-Net that outputs the predicted noise.

In [ ]:
class Diffusion:    def __init__(self, timesteps: int = TIMESTEPS, device: torch.device = DEVICE):        self.timesteps = timesteps        self.device = device        betas = torch.linspace(1e-4, 0.02, timesteps, dtype=torch.float32)        alphas = 1.0 - betas        alpha_cumprod = torch.cumprod(alphas, dim=0)        self.betas = betas.to(device)        self.alphas = alphas.to(device)        self.alpha_cumprod = alpha_cumprod.to(device)    def q_sample(self, x0: torch.Tensor, t: torch.Tensor, noise: torch.Tensor = None):        """Sample x_t from q(x_t | x0)."""        if noise is None:            noise = torch.randn_like(x0)        sqrt_alpha_bar = torch.sqrt(self.alpha_cumprod[t]).view(-1, 1, 1, 1)        sqrt_one_minus = torch.sqrt(1 - self.alpha_cumprod[t]).view(-1, 1, 1, 1)        return sqrt_alpha_bar * x0 + sqrt_one_minus * noisedef sinusoidal_embedding(timesteps: torch.Tensor, dim: int = 128):    device = timesteps.device    half_dim = dim // 2    emb = math.log(10000) / (half_dim - 1)    emb = torch.exp(torch.arange(half_dim, device=device) * -emb)    emb = timesteps.float().unsqueeze(1) * emb.unsqueeze(0)    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)    if dim % 2 == 1:        emb = torch.nn.functional.pad(emb, (0, 1))    return embclass ConvBlock(nn.Module):    def __init__(self, in_ch, out_ch, time_dim):        super().__init__()        self.block = nn.Sequential(            nn.Conv2d(in_ch, out_ch, 3, padding=1),            nn.GroupNorm(8, out_ch),            nn.ReLU(inplace=True),            nn.Conv2d(out_ch, out_ch, 3, padding=1),            nn.GroupNorm(8, out_ch),            nn.ReLU(inplace=True),        )        self.time_mlp = nn.Sequential(            nn.Linear(time_dim, out_ch),            nn.ReLU(inplace=True)        )    def forward(self, x, t):        h = self.block(x)        time_emb = self.time_mlp(t)[:, :, None, None]        return h + time_embclass Down(nn.Module):    def __init__(self, in_ch, out_ch, time_dim):        super().__init__()        self.conv = ConvBlock(in_ch, out_ch, time_dim)        self.pool = nn.MaxPool2d(2)    def forward(self, x, t):        x = self.conv(x, t)        p = self.pool(x)        return x, pclass Up(nn.Module):    def __init__(self, in_ch, out_ch, time_dim):        super().__init__()        self.conv = ConvBlock(in_ch + out_ch, out_ch, time_dim)    def forward(self, x, skip, t):        x = nn.functional.interpolate(x, scale_factor=2, mode="nearest")        x = torch.cat([x, skip], dim=1)        return self.conv(x, t)class SimpleUNet(nn.Module):    def __init__(self, img_channels=1, time_dim=128):        super().__init__()        in_ch = img_channels + 1  # extra channel for mask        self.time_dim = time_dim        self.inc = ConvBlock(in_ch, 32, time_dim)        self.down1 = Down(32, 64, time_dim)        self.down2 = Down(64, 128, time_dim)        self.bot = ConvBlock(128, 128, time_dim)        self.up2 = Up(128, 128, time_dim)        self.up1 = Up(128, 64, time_dim)        self.outc = nn.Sequential(            nn.Conv2d(64, 32, 3, padding=1),            nn.ReLU(inplace=True),            nn.Conv2d(32, img_channels, 1)        )    def forward(self, x, t):        t_emb = sinusoidal_embedding(t, self.time_dim)        x1 = self.inc(x, t_emb)        x2, p1 = self.down1(x1, t_emb)        x3, p2 = self.down2(x2, t_emb)        bottleneck = self.bot(p2, t_emb)        x = self.up2(bottleneck, x3, t_emb)        x = self.up1(x, x2, t_emb)        return self.outc(x)

## Training loopTraining samples a random timestep `t` and noise `\epsilon ~ N(0, I)`, forms `x_t = q(x_t | x_0, t, \epsilon)`, and builds a masked version that removes the original signal inside the hole:\[ x^{	ext{masked}}_t = x_t (1-	ext{mask}) + \sqrt{1-ar{lpha}_t}\, \epsilon \cdot 	ext{mask}. \]This keeps the unmasked area consistent with `x_t` while ensuring the masked area contains only the same noise used in `q_sample`, eliminating leakage of `x_0`. The old code incorrectly injected fresh noise inside the mask, forcing the model to predict noise different from its input. Here the identical noise tensor is used for both `q_sample` and constructing `x^{	ext{masked}}_t`, so the supervised target matches the visible input statistics. The loss emphasises the masked region while keeping a global term.

In [ ]:
diffusion = Diffusion(timesteps=TIMESTEPS, device=DEVICE)model = SimpleUNet(img_channels=CHANNELS, time_dim=128).to(DEVICE)optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)mse_loss = nn.MSELoss()def train_epoch(loader):    model.train()    epoch_loss = 0.0    for imgs in tqdm(loader, leave=False):        imgs = imgs.to(DEVICE)        _, mask = apply_inpainting_mask(imgs, ratio=MASK_RATIO)        optimizer.zero_grad()        b = imgs.size(0)        t = torch.randint(0, diffusion.timesteps, (b,), device=DEVICE)        noise = torch.randn_like(imgs)        x_t = diffusion.q_sample(imgs, t, noise)        alpha_bar_t = diffusion.alpha_cumprod[t].view(b, 1, 1, 1)        sqrt_one_minus_ac = torch.sqrt(1 - alpha_bar_t)        # Remove the clean component inside the mask; keep unmasked region intact        x_t_masked = x_t * (1 - mask) + sqrt_one_minus_ac * noise * mask        net_inp = torch.cat([x_t_masked, mask], dim=1)        pred_noise = model(net_inp, t)        masked_loss = F.mse_loss(pred_noise * mask, noise * mask, reduction="sum") / (mask.sum() + 1e-8)        global_loss = mse_loss(pred_noise, noise)        loss = 0.7 * masked_loss + 0.3 * global_loss        loss.backward()        optimizer.step()        epoch_loss += loss.item()    return epoch_loss / len(loader)def train(epochs=EPOCHS):    losses = []    for epoch in range(1, epochs + 1):        loss = train_epoch(loader)        losses.append(loss)        print(f"Epoch {epoch:03d} | loss: {loss:.4f}")    return losses# Uncomment to train (may take a few minutes depending on hardware)# training_losses = train()

## Inpainting and evaluationTo inpaint, we start from random noise and iterate the reverse diffusion steps. At each timestep we:1. Hide the masked region with fresh noise and keep the known pixels.2. Predict the noise with the U-Net and compute \(x_{t-1}\) using the DDPM reverse formula.3. Enforce the known pixels outside the mask to match the original image.4. Blend the update with the predicted clean image using a guidance weight.We visualise the original, masked, and inpainted images, and optionally report the mean squared error inside the masked region.

In [ ]:
@torch.no_grad()def p_sample_step(x, t, eps_pred, diffusion: Diffusion):    beta_t = diffusion.betas[t]    alpha_t = diffusion.alphas[t]    alpha_bar_t = diffusion.alpha_cumprod[t]    sqrt_one_minus_ab = torch.sqrt(1 - alpha_bar_t)    sqrt_recip_alpha = torch.sqrt(1 / alpha_t)    coeff = (1 - alpha_t) / sqrt_one_minus_ab    mean = sqrt_recip_alpha * (x - coeff * eps_pred)    if t == 0:        return mean    noise = torch.randn_like(x)    return mean + torch.sqrt(beta_t) * noise@torch.no_grad()def inpaint(imgs: torch.Tensor, ratio: float = MASK_RATIO, steps: int = TIMESTEPS, guide_strength: float = 0.3):    model.eval()    imgs = imgs.to(DEVICE)    b = imgs.size(0)    masked_imgs, mask = apply_inpainting_mask(imgs, ratio=ratio)    x = torch.randn_like(imgs)  # start from pure noise    for step in reversed(range(steps)):        t = torch.full((b,), step, device=DEVICE, dtype=torch.long)        alpha_bar_t = diffusion.alpha_cumprod[t].view(b, 1, 1, 1)        sqrt_one_minus_ab = torch.sqrt(1 - alpha_bar_t)        # Hide the target region with matching noise statistics        masked_noise = torch.randn_like(x) * sqrt_one_minus_ab        x_masked = x * (1 - mask) + masked_noise * mask        net_inp = torch.cat([x_masked, mask], dim=1)        eps_pred = model(net_inp, t)        x0_pred = (x - sqrt_one_minus_ab * eps_pred) / torch.sqrt(alpha_bar_t)        x = p_sample_step(x, step, eps_pred, diffusion)        # Keep known pixels close to the original        x = x * mask + imgs * (1 - mask)        # Blend with predicted clean image        x = (1 - guide_strength) * x + guide_strength * x0_pred    return x.clamp(-1, 1), masked_imgs, maskdef denorm(x):    return (x + 1) / 2def visualize_inpainting(batch):    inpainted, masked_imgs, _ = inpaint(batch, ratio=MASK_RATIO, steps=TIMESTEPS)    batch = batch.cpu()    masked_imgs = masked_imgs.cpu()    inpainted = inpainted.cpu()    fig, axes = plt.subplots(3, 6, figsize=(12, 6))    for i in range(6):        axes[0, i].imshow(denorm(batch[i]).squeeze(), cmap="gray", vmin=0, vmax=1)        axes[0, i].axis("off")        axes[0, i].set_title("Original")        axes[1, i].imshow(denorm(masked_imgs[i]).squeeze(), cmap="gray", vmin=0, vmax=1)        axes[1, i].axis("off")        axes[1, i].set_title("Masked")        axes[2, i].imshow(denorm(inpainted[i]).squeeze(), cmap="gray", vmin=0, vmax=1)        axes[2, i].axis("off")        axes[2, i].set_title("Inpainted")    plt.tight_layout()    plt.show()def masked_mse(inpainted, original, mask):    return F.mse_loss(inpainted * mask, original * mask)# Example usage (requires a trained model)# visualize_inpainting(next(iter(loader)))

## ConclusionA compact diffusion model paired with a U-Net can inpaint missing regions in simple shape images. The critical fix was to ensure the same noise tensor is used both for generating \(x_t\) and for constructing the masked input; previously, unrelated noise inside the mask prevented learning. Visual inspection of masked versus reconstructed images, along with masked-region MSE, shows the model can recover the withheld central area after training.